<a href="https://colab.research.google.com/github/Adrianna40/fintech/blob/main/credit_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h2> Give Me Some Credit Kaggle Competition </h2> </center>
<center> <i>  Seena Huang, Cecelia Sanborn, Harry Bendekgey, Simon Posada Fishman </i> </center>
<center> <a href="https://www.kaggle.com/c/GiveMeSomeCredit">https://www.kaggle.com/c/GiveMeSomeCredit<a/> </center>

In [ ]:
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

import time
import os

In [ ]:
df = pd.read_csv('https://students.mimuw.edu.pl/~kr394714/FinTech/cs-training.csv')
list(df)

In [ ]:
df.SeriousDlqin2yrs.mean()

In [ ]:
age_hist = df['age'].hist(by=df['SeriousDlqin2yrs'], bins=20, layout=(2,1))
age_hist[0].set_xlim((0,100))
age_hist[0].set_title('SeriousDlqin2yrs = 0')
age_hist[1].set_xlim((0,100))
age_hist[1].set_title('SeriousDlqin2yrs = 1')

In [ ]:
df.DebtRatio.describe()

In [ ]:
df.DebtRatio.quantile([.975])

In [ ]:
df[df['DebtRatio'] > 3489.025][['SeriousDlqin2yrs','MonthlyIncome']].describe()

In [ ]:
df[(df['DebtRatio'] > 3489.025) & (df['SeriousDlqin2yrs'] == df['MonthlyIncome'])].shape[0]

In [ ]:
df.groupby('NumberOfTimes90DaysLate').NumberOfTimes90DaysLate.count()

In [ ]:
df[df['NumberOfTimes90DaysLate'] > 95][['SeriousDlqin2yrs','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTimes90DaysLate']].describe()

In [ ]:
df[(df['RevolvingUtilizationOfUnsecuredLines'] > .9) & (df['RevolvingUtilizationOfUnsecuredLines'] <= 4)].SeriousDlqin2yrs.describe()

In [ ]:
df[(df['RevolvingUtilizationOfUnsecuredLines'] > 4) & (df['RevolvingUtilizationOfUnsecuredLines'] <= 10)].SeriousDlqin2yrs.describe()

In [ ]:
df[df['RevolvingUtilizationOfUnsecuredLines'] > 10].describe()

In [ ]:
not_missing = df.dropna()
target = 'MonthlyIncome'
predictors = [c for c in list(not_missing) if c not in [target, 'Unnamed: 0','SeriousDlqin2yrs']]
X_data = not_missing[predictors]
y_data = not_missing[target]
regr = LinearRegression().fit(X_data, y_data)

In [ ]:
regr.score(X_data, y_data)

## Datasets

In [ ]:
# Median Fill, Outliers Removed
removed_debt_outliers = df.drop(df[df['DebtRatio'] > 3489.025].index)
removed_debt_outliers = removed_debt_outliers.fillna(removed_debt_outliers.median())

In [ ]:
# Removed utilization outliers
dfus = removed_debt_outliers.drop(removed_debt_outliers[removed_debt_outliers['RevolvingUtilizationOfUnsecuredLines'] > 10].index)

In [ ]:
# Removed 98s
dfn98 = dfus.copy()
dfn98.loc[dfn98['NumberOfTime30-59DaysPastDueNotWorse'] > 90, 'NumberOfTime30-59DaysPastDueNotWorse'] = 18
dfn98.loc[dfn98['NumberOfTime60-89DaysPastDueNotWorse'] > 90, 'NumberOfTime60-89DaysPastDueNotWorse'] = 18
dfn98.loc[dfn98['NumberOfTimes90DaysLate'] > 90, 'NumberOfTimes90DaysLate'] = 18

# Model Testing

In [ ]:
# A utility class to test all of our models on different datasets
class Tester():
    def __init__(self, target):
        self.target = target
        self.datasets = {}
        self.models = {}
        self.cache = {} # we added a simple cache to speed things up

    def addDataset(self, name, df):
        self.datasets[name] = df.copy()

    def addModel(self, name, model):
        self.models[name] = model
        
    def clearModels(self):
        self.models = {}

    def clearCache(self):
        self.cache = {}
    
    def testModelWithDataset(self, m_name, df_name, sample_len, cv):
        if (m_name, df_name, sample_len, cv) in self.cache:
            return self.cache[(m_name, df_name, sample_len, cv)]

        clf = self.models[m_name]
        
        if not sample_len: 
            sample = self.datasets[df_name]
        else: sample = self.datasets[df_name].sample(sample_len)
            
        X = sample.drop([self.target], axis=1)
        Y = sample[self.target]

        s = cross_validate(clf, X, Y, scoring=['roc_auc'], cv=cv, n_jobs=-1)
        self.cache[(m_name, df_name, sample_len, cv)] = s

        return s

    def runTests(self, sample_len=80000, cv=4):
        # Tests the added models on all the added datasets
        scores = {}
        for m_name in self.models:
            for df_name in self.datasets:
                # print('Testing %s' % str((m_name, df_name)), end='')
                start = time.time()

                score = self.testModelWithDataset(m_name, df_name, sample_len, cv)
                scores[(m_name, df_name)] = score
                
                end = time.time()
                
                # print(' -- %0.2fs ' % (end - start))

        print('--- Top 10 Results ---')
        for score in sorted(scores.items(), key=lambda x: -1 * x[1]['test_roc_auc'].mean())[:10]:
            auc = score[1]['test_roc_auc']
            print("%s --> AUC: %0.4f (+/- %0.4f)" % (str(score[0]), auc.mean(), auc.std()))

            
# We will use a tester object across all models
tester = Tester('SeriousDlqin2yrs')

# You can add datasets like this:
tester.addDataset('Drop Missing', df.dropna())

# And models like this:
rfc = RandomForestClassifier(n_estimators=15, max_depth = 6, random_state=0)
tester.addModel('Simple Random Forest', rfc)
tester.addModel('Simple SVM', svm.LinearSVC())

# You can then use it to run the tests
tester.runTests()

--- Top 10 Results ---
('Simple Random Forest', 'Drop Missing') --> AUC: 0.8487 (+/- 0.0075)
('Simple SVM', 'Drop Missing') --> AUC: 0.5819 (+/- 0.0708)


In [ ]:
tester.addDataset('Median Fill', df.fillna(df.median()))
tester.addDataset('Median Fill, Outliers Removed', removed_debt_outliers)
tester.addDataset('Removed 98s', dfn98)
tester.addDataset('Removed utilization outliers', dfus)

tester.runTests()

--- Top 10 Results ---
('Simple Random Forest', 'Removed 98s') --> AUC: 0.8592 (+/- 0.0033)
('Simple Random Forest', 'Median Fill, Outliers Removed') --> AUC: 0.8578 (+/- 0.0021)
('Simple Random Forest', 'Removed utilization outliers') --> AUC: 0.8574 (+/- 0.0041)
('Simple Random Forest', 'Median Fill') --> AUC: 0.8573 (+/- 0.0011)
('Simple Random Forest', 'Drop Missing') --> AUC: 0.8487 (+/- 0.0075)
('Simple SVM', 'Removed utilization outliers') --> AUC: 0.6287 (+/- 0.0234)
('Simple SVM', 'Median Fill, Outliers Removed') --> AUC: 0.6061 (+/- 0.0524)
('Simple SVM', 'Removed 98s') --> AUC: 0.6042 (+/- 0.0240)
('Simple SVM', 'Median Fill') --> AUC: 0.5844 (+/- 0.0276)
('Simple SVM', 'Drop Missing') --> AUC: 0.5819 (+/- 0.0708)


## Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

for i in range(5,10):
    for j in range(10,20):
        rfc = RandomForestClassifier(n_estimators=j,max_depth = i, random_state=0)
        tester.addModel('Random Forest '+'d: '+str(i)+' est: '+str(j)  ,rfc)

tester.runTests()

--- Top 10 Results ---
('Random Forest d: 8 est: 17', 'Median Fill, Outliers Removed') --> AUC: 0.8649 (+/- 0.0044)
('Random Forest d: 9 est: 11', 'Removed utilization outliers') --> AUC: 0.8648 (+/- 0.0033)
('Random Forest d: 9 est: 16', 'Median Fill, Outliers Removed') --> AUC: 0.8643 (+/- 0.0054)
('Random Forest d: 7 est: 11', 'Removed 98s') --> AUC: 0.8637 (+/- 0.0065)
('Random Forest d: 9 est: 11', 'Median Fill, Outliers Removed') --> AUC: 0.8636 (+/- 0.0047)
('Random Forest d: 6 est: 19', 'Removed utilization outliers') --> AUC: 0.8634 (+/- 0.0037)
('Random Forest d: 9 est: 17', 'Removed utilization outliers') --> AUC: 0.8633 (+/- 0.0023)
('Random Forest d: 8 est: 10', 'Median Fill, Outliers Removed') --> AUC: 0.8633 (+/- 0.0024)
('Random Forest d: 9 est: 14', 'Removed utilization outliers') --> AUC: 0.8632 (+/- 0.0045)
('Random Forest d: 9 est: 13', 'Median Fill, Outliers Removed') --> AUC: 0.8631 (+/- 0.0022)


## K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

tester.clearModels()

for i in range(5, 10):
    neigh = KNeighborsClassifier(n_neighbors=i)
    tester.addModel('KNN k=%d' % i, neigh)
    
tester.runTests(50000)

--- Top 10 Results ---
('KNN k=7', 'Removed utilization outliers') --> AUC: 0.5257 (+/- 0.0074)
('KNN k=9', 'Median Fill') --> AUC: 0.5251 (+/- 0.0021)
('KNN k=8', 'Removed utilization outliers') --> AUC: 0.5240 (+/- 0.0042)
('KNN k=8', 'Median Fill') --> AUC: 0.5195 (+/- 0.0100)
('KNN k=9', 'Removed 98s') --> AUC: 0.5190 (+/- 0.0111)
('KNN k=7', 'Median Fill') --> AUC: 0.5187 (+/- 0.0082)
('KNN k=5', 'Median Fill') --> AUC: 0.5176 (+/- 0.0067)
('KNN k=9', 'Median Fill, Outliers Removed') --> AUC: 0.5160 (+/- 0.0080)
('KNN k=7', 'Drop Missing') --> AUC: 0.5151 (+/- 0.0104)
('KNN k=5', 'Median Fill, Outliers Removed') --> AUC: 0.5149 (+/- 0.0049)
